# NLP Tweet Sentiment Analysis

### Projektziel
Dieses Projekt zielt darauf ab, Tweets in positive und negative Kategorien zu klassifizieren.  
Es verwendet moderne NLP-Techniken wie:
- **TF-IDF**: Zur numerischen Darstellung von Textdaten.
- **GloVe**: Vortrainierte Embeddings zur besseren Erfassung von Wortbeziehungen.
- **Emoji2Vec**: Speziell zur Erkennung von Emoji-Bedeutungen.
- **BERT**: Kontextuelle Embeddings für tiefere semantische Analysen.

### Methodik
1. **Datenaufbereitung**:
   - Bereinigung der Tweets (z. B. Entfernung von URLs und Mentions).
   - Tokenisierung, Stemming und Lemmatization.
2. **Feature Engineering**:
   - Nutzung von TF-IDF, GloVe, Emoji2Vec und BERT.
   - Kombination der verschiedenen Features in einer skalierbaren Repräsentation.
3. **Modelltraining**:
   - Einsatz von LightGBM, einem leistungsstarken Algorithmus für tabellarische Daten.
4. **Bewertung**:
   - Analyse der Modellleistung mit Genauigkeit, F1-Score und Confusion-Matrix.

### Datenquelle
Die Daten stammen aus dem **NLTK Twitter-Datensatz**, der aus positiven und negativen Tweets besteht:
- **Positive Tweets**: Glückliche oder optimistische Inhalte.
- **Negative Tweets**: Kritische oder wütende Inhalte.

### Ziel
Erreichen einer Modellgenauigkeit von mindestens **90 %** auf den Testdaten.

In [ ]:
import sys
import os

# Korrigiere den Modulpfad, um zum `src`-Verzeichnis zu gelangen
module_path = os.path.abspath(os.path.join("..", "src"))  # Gehe ein Verzeichnis zurück und dann in "src"
if module_path not in sys.path:
    sys.path.append(module_path)


# Daten laden 
from dataset_preparation import load_twitter_data

# Positive und negative Tweets laden
positive_tweets, negative_tweets = load_twitter_data()

# Überblick über die Daten
print(f"Anzahl positiver Tweets: {len(positive_tweets)}")
print(f"Anzahl negativer Tweets: {len(negative_tweets)}")

# Beispiele anzeigen
print("\nBeispiel positiver Tweet:")
print(positive_tweets[0])
print("\nBeispiel negativer Tweet:")
print(negative_tweets[0])


### 2. Importe und Ressourcen

In diesem Schritt werden alle benötigten Bibliotheken und Ressourcen importiert.  
Zusätzlich werden die vortrainierten Modelle (GloVe, Emoji2Vec und BERT) vorbereitet, um sie später in der Pipeline zu verwenden.

- **NLTK**: Für Textverarbeitung wie Tokenisierung und Stoppwortentfernung.
- **TF-IDF**: Vektorisierung des Texts.
- **GloVe**: Vortrainierte Wort-Embeddings.
- **Emoji2Vec**: Spezielle Vektoren für Emojis.
- **BERT**: Kontextuelle Wort-Embeddings.

In [ ]:
# Bibliotheken importieren
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from transformers import AutoTokenizer, AutoModel
from lightgbm import LGBMClassifier

# Eigene Module importieren
from dataset_preparation import load_twitter_data, split_data
from preprocess import (
    preprocess_tweet,
    vectorize_with_tfidf,
    load_glove_embeddings,
    vectorize_with_glove,
    load_emoji2vec,
    vectorize_with_emojis,
    load_bert_model,
    vectorize_with_bert,
)
from train_model import train_lightgbm, evaluate_model, plot_confusion_matrix

# Ressourcen für NLP vorbereiten
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# Ressourcen initialisieren
print("Alle Ressourcen und Bibliotheken erfolgreich importiert.")


### 3. Preprocessing

Das Preprocessing umfasst die Bereinigung und Tokenisierung der Tweets, gefolgt von der Anwendung von NLP-Techniken zur Erstellung von Features:

1. **Bereinigung**:
   - Entfernen von URLs, Mentions und Sonderzeichen.
   - Tokenisierung der Texte in einzelne Wörter.

2. **Linguistische Verarbeitung**:
   - Entfernen von Stopwörtern (z. B. "and", "the").
   - Lemmatisierung, um Wörter auf ihre Grundform zu reduzieren (z. B. "running" → "run").

3. **Feature-Vektorisierung**:
   - **TF-IDF**: Zuweisung von Gewichtungen basierend auf der Häufigkeit eines Wortes in einem Dokument und im gesamten Korpus.
   - **GloVe**: Vortrainierte Wort-Embeddings, die semantische Beziehungen zwischen Wörtern berücksichtigen.
   - **Emoji2Vec**: Spezielle Embeddings für Emojis.
   - **BERT**: Kontextuelle Embeddings, die die Bedeutung eines Wortes basierend auf seinem Kontext erfassen.

In [ ]:
# Daten aufteilen
from dataset_preparation import split_data

print("Teile Daten in Training, Validierung und Test auf...")
X_train, X_valid, X_test, y_train, y_valid, y_test = split_data(
    positive_tweets, negative_tweets, balance=True
)

print(f"Train Size: {len(X_train)}, Valid Size: {len(X_valid)}, Test Size: {len(X_test)}")
print(f"Training: {sum(y_train)} positive, {len(y_train) - sum(y_train)} negative")
print(f"Validation: {sum(y_valid)} positive, {len(y_valid) - sum(y_valid)} negative")
print(f"Test: {sum(y_test)} positive, {len(y_test) - sum(y_test)} negative")


# Vorverarbeitung der Tweets
print("Starte Preprocessing...")

# Bereinigung und Tokenisierung
X_train_preprocessed = [preprocess_tweet(tweet) for tweet in X_train]
X_valid_preprocessed = [preprocess_tweet(tweet) for tweet in X_valid]
X_test_preprocessed = [preprocess_tweet(tweet) for tweet in X_test]

print("Preprocessing abgeschlossen.")

# Beispiel-Tweets nach Preprocessing
print("\nBeispiel-Tweet vor und nach Preprocessing:")
print(f"Original: {X_train[0]}")
print(f"Preprocessed: {X_train_preprocessed[0]}")

### 4. Datenaufteilung

In diesem Schritt werden die geladenen Tweets in drei Datensätze aufgeteilt:

1. **Training**:
   - Für das Training des Modells verwendet.
2. **Validierung**:
   - Für die Hyperparameter-Optimierung und zur Bewertung während des Trainings.
3. **Test**:
   - Für die abschließende Bewertung der Modellleistung.

### **Stratifizierte Aufteilung**
Die Aufteilung stellt sicher, dass die Klassenverteilung in allen Datensätzen konsistent bleibt (positive und negative Tweets sind gleichmäßig verteilt).

### **Balance der Daten**
Falls die Klassenverteilung unausgewogen ist, wird ein Oversampling angewandt, um sicherzustellen, dass beide Klassen im Trainingsdatensatz gleich stark vertreten sind.

In [ ]:
# Datenaufteilung
from dataset_preparation import load_twitter_data, split_data

# Tweets laden
positive_tweets, negative_tweets = load_twitter_data()

# Aufteilen der Daten
print("Teile Daten in Training, Validierung und Test auf...")
X_train, X_valid, X_test, y_train, y_valid, y_test = split_data(
    positive_tweets, negative_tweets, balance=True
)

# Überblick über die Datengröße
print(f"Train Size: {len(X_train)}, Valid Size: {len(X_valid)}, Test Size: {len(X_test)}")
print(f"Training: {sum(y_train)} positive, {len(y_train) - sum(y_train)} negative")
print(f"Validation: {sum(y_valid)} positive, {len(y_valid) - sum(y_valid)} negative")
print(f"Test: {sum(y_test)} positive, {len(y_test) - sum(y_test)} negative")

# Beispiel eines Tweets und Labels
print("\nBeispiel-Tweets aus Trainingsdaten:")
print(f"Tweet: {X_train[0]}, Label: {y_train[0]}")

### 5. Feature-Engineering

Im Feature-Engineering werden die Tweets in numerische Vektoren umgewandelt, die das Modell als Eingabe akzeptiert.  
Dafür werden folgende Ansätze verwendet:

1. **TF-IDF**:
   - Erzeugt numerische Repräsentationen basierend auf der Häufigkeit von Wörtern.
   - Erfasst Uni- und Bigramme mit einer maximalen Anzahl von 20.000 Features.

2. **GloVe**:
   - Verwendet vortrainierte Wort-Embeddings, die semantische Beziehungen zwischen Wörtern berücksichtigen.
   - Tweets werden durch den Mittelwert der Embeddings aller enthaltenen Wörter dargestellt.

3. **Emoji2Vec**:
   - Spezialisierte Embeddings zur Erkennung von Bedeutungen von Emojis.
   - Tweets werden durch den Mittelwert der Vektoren der enthaltenen Emojis dargestellt.

4. **BERT**:
   - Kontextuelle Embeddings, die die Bedeutung eines Wortes basierend auf seinem Kontext im Satz erfassen.
   - Tweets werden durch den Mittelwert aller Tokens des Modells dargestellt.

Alle erzeugten Features werden später kombiniert und skaliert, um sie für das Modelltraining vorzubereiten.

In [ ]:
import os
# Feature-Engineering vorbereiten
from preprocess import (
    vectorize_with_tfidf,
    load_glove_embeddings,
    vectorize_with_glove,
    load_emoji2vec,
    vectorize_with_emojis,
    load_bert_model,
    vectorize_with_bert,
)

# Prüfen, ob das Arbeitsverzeichnis bereits korrekt ist
current_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(current_dir, "./"))

if current_dir != project_dir:
    os.chdir(project_dir)

print("Aktuelles Arbeitsverzeichnis:", os.getcwd())

# Pfade zu GloVe und Emoji2Vec
glove_path = "./data/glove.twitter.27B.200d.txt"
emoji2vec_path = "./data/emoji2vec.txt"


# GloVe-Embeddings laden
print("Lade GloVe-Embeddings...")
glove_embeddings, glove_mean = load_glove_embeddings(glove_path)

# Emoji2Vec-Embeddings laden
print("Lade Emoji2Vec-Embeddings...")
emoji_vectors = load_emoji2vec(emoji2vec_path)

# BERT-Modell laden
print("Lade BERT-Modell...")
tokenizer, bert_model = load_bert_model("distilbert-base-uncased")

# TF-IDF-Vektorisierung
print("Vektorisieren mit TF-IDF...")
tfidf_matrix_train, tfidf_vectorizer = vectorize_with_tfidf(X_train_preprocessed)
tfidf_matrix_valid = tfidf_vectorizer.transform(X_valid_preprocessed)
tfidf_matrix_test = tfidf_vectorizer.transform(X_test_preprocessed)

# GloVe-Vektorisierung
print("Vektorisieren mit GloVe...")
glove_matrix_train = vectorize_with_glove(X_train_preprocessed, glove_embeddings, glove_mean)
glove_matrix_valid = vectorize_with_glove(X_valid_preprocessed, glove_embeddings, glove_mean)
glove_matrix_test = vectorize_with_glove(X_test_preprocessed, glove_embeddings, glove_mean)

# Emoji2Vec-Vektorisierung
print("Vektorisieren mit Emoji2Vec...")
emoji_matrix_train = vectorize_with_emojis(X_train_preprocessed, emoji_vectors)
emoji_matrix_valid = vectorize_with_emojis(X_valid_preprocessed, emoji_vectors)
emoji_matrix_test = vectorize_with_emojis(X_test_preprocessed, emoji_vectors)

# BERT-Vektorisierung
print("Vektorisieren mit BERT...")
bert_matrix_train = vectorize_with_bert(X_train, tokenizer, bert_model)
bert_matrix_valid = vectorize_with_bert(X_valid, tokenizer, bert_model)
bert_matrix_test = vectorize_with_bert(X_test, tokenizer, bert_model)

print("Feature-Engineering abgeschlossen.")


### 6. Modelltraining

In diesem Schritt wird ein **LightGBM-Modell** trainiert, das für tabellarische Daten besonders gut geeignet ist.  

#### **Einstellungen des Modells**:
- **Random State**: Für Reproduzierbarkeit.
- **Class Weight**: "Balanced", um Ungleichgewichte zwischen Klassen auszugleichen.
- **Hyperparameter**:
  - `n_estimators=500`: Anzahl der Entscheidungsbäume.
  - `max_depth=20`: Maximale Tiefe der Bäume.

#### **Ablauf**:
1. **Feature-Kombination**:
   - TF-IDF, GloVe, Emoji2Vec und BERT-Features werden zu einem einzigen Feature-Set kombiniert.
2. **Skalierung**:
   - Die kombinierten Features werden standardisiert, um die Verteilung auszugleichen.
3. **Training**:
   - Das Modell wird mit den skalierten Features trainiert.

In [ ]:
from sklearn.preprocessing import StandardScaler
from train_model import train_lightgbm, evaluate_model, plot_confusion_matrix

# Feature-Kombination
print("Kombiniere Features...")
combined_train = np.hstack([tfidf_matrix_train.toarray(), glove_matrix_train, emoji_matrix_train, bert_matrix_train])
combined_valid = np.hstack([tfidf_matrix_valid.toarray(), glove_matrix_valid, emoji_matrix_valid, bert_matrix_valid])
combined_test = np.hstack([tfidf_matrix_test.toarray(), glove_matrix_test, emoji_matrix_test, bert_matrix_test])

# Skalierung der Features
print("Skaliere Features...")
scaler = StandardScaler()
combined_train_scaled = scaler.fit_transform(combined_train)
combined_valid_scaled = scaler.transform(combined_valid)
combined_test_scaled = scaler.transform(combined_test)

# Modelltraining
print("Trainiere LightGBM-Modell...")
model = train_lightgbm(combined_train_scaled, y_train)

# Modellbewertung
print("Bewertung des Modells...")
accuracy, conf_matrix, report = evaluate_model(model, combined_test_scaled, y_test)

# Ergebnisse anzeigen
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)
plot_confusion_matrix(conf_matrix)

### 7. Modellbewertung

Nach dem Training wird das Modell mit den Testdaten bewertet, um die endgültige Leistung zu messen.  
Die Bewertung umfasst:

1. **Testgenauigkeit (Accuracy)**:
   - Gibt an, wie viele der Vorhersagen korrekt sind.

2. **Classification Report**:
   - **Precision**: Anteil der korrekten positiven Vorhersagen an allen positiven Vorhersagen.
   - **Recall**: Anteil der korrekt identifizierten positiven Instanzen an allen tatsächlichen positiven Instanzen.
   - **F1-Score**: Harmonisches Mittel aus Precision und Recall.

3. **Confusion Matrix**:
   - Zeigt die Anzahl der True Positives, True Negatives, False Positives und False Negatives.

#### **Ziel**:
- Überprüfung, ob die Genauigkeit mindestens **90 %** beträgt.
- Analyse der Fehlklassifikationen für potenzielle Verbesserungen.

In [ ]:
# Bewertungsergebnisse anzeigen
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)

# Confusion Matrix visualisieren
plot_confusion_matrix(conf_matrix)

# Zusätzliche Analyse (optional)
false_positives = (conf_matrix[0][1])
false_negatives = (conf_matrix[1][0])
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")

### 8. Fazit

#### **Ergebnisse**
- **Genauigkeit**: Das Modell erreichte eine Genauigkeit von **96.40 %** auf den Testdaten, was die ursprüngliche Zielvorgabe von **90 %** übertrifft.
- **Starke Leistung**:
  - **Precision und Recall** sind für beide Klassen (positive und negative Tweets) ausgeglichen.
  - **F1-Score** zeigt, dass das Modell robust und zuverlässig arbeitet.

#### **Verwendete Technologien**
1. **TF-IDF**: Bewährter Ansatz zur numerischen Repräsentation von Textdaten.
2. **GloVe und Emoji2Vec**: Ergänzen sich durch semantische und emoji-spezifische Features.
3. **BERT**: Kontextuelle Embeddings verbessern die Modellleistung signifikant.
4. **LightGBM**: Effektives Modell für tabellarische Daten, das mit wenig Aufwand optimiert werden kann.

#### **Herausforderungen**
1. **Feature-Kombination**:
   - Die Kombination mehrerer Embedding-Methoden erhöht die Komplexität.
2. **Speicherverbrauch**:
   - Die Verwendung großer vortrainierter Modelle wie BERT kann speicherintensiv sein.
3. **Datenverarbeitung**:
   - Die Bereinigung und Verarbeitung der Tweets ist zeitaufwändig.

#### **Mögliche Verbesserungen**
1. **Hyperparameter-Tuning**:
   - Optimierung des LightGBM-Modells, um die Leistung weiter zu steigern.
2. **Datenaugmentation**:
   - Hinzufügen weiterer Trainingsdaten durch Augmentationstechniken.
3. **Experimentieren mit anderen Modellen**:
   - Testen von Modellen wie XGBoost oder Deep Learning-Ansätzen.

#### **Zusammenfassung**
Das Projekt demonstriert, wie moderne NLP-Techniken kombiniert werden können, um ein robustes Modell zur Sentiment-Analyse von Tweets zu erstellen. Die Erzielung einer Genauigkeit von über 96 % zeigt die Stärke des Ansatzes.